# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

In [3]:
import os
from openai import OpenAI

llm_model = "gpt-3.5-turbo"

# Load openai api key
OPENAI_API_KEY_PATH = '/home/loc/Documents/OPENAI_API_KEY.txt'
with open(OPENAI_API_KEY_PATH) as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# ConversationBufferMemory

*Note*: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [4]:
llm = ChatOpenAI(temperature = 0.0, model = llm_model)

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [5]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Is there anything else you would like to know, Andrew?'

In [8]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew. Is there anything else you would like to know, Andrew?


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew. Is there anything else you would like to know, Andrew?"}

In [10]:
memory = ConversationBufferMemory()

In [12]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [13]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

# ConversationBufferWindowMemory

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [19]:
memory = ConversationBufferWindowMemory(k=1)               

In [20]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [21]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [22]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [23]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How are you doing today?"

In [24]:
conversation.predict(input="What is 1+1?")

'1 + 1 equals 2. Is there anything else you would like to know or talk about?'

In [25]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know or talk about?"

In [26]:
# ConversationTokenBufferMemory

In [35]:
from langchain.memory import ConversationTokenBufferMemory

In [36]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})

memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})

memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [37]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})

memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})

memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

# ConversationSummaryMemory

In [38]:
from langchain.memory import ConversationSummaryBufferMemory

In [39]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

memory.save_context({"input": "Hello"}, {"output": "What's up"})

memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to showcase the latest LLM demo during lunch."}

In [40]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [41]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to showcase the latest LLM demo during lunch.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the LLM demo, you could showcase the latest language model capabilities, such as text generation, language translation, sentiment analysis, and question-answering. You could also highlight the model's accuracy, speed, and efficiency compared to other language models in the market. Additionally, you could demonstrate how the model can be fine-tuned for specific tasks or industries to showcase its versatility and customization options."

In [42]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to showcase the latest LLM demo during lunch. The human asks what would be a good demo to show.\nAI: For the LLM demo, you could showcase the latest language model capabilities, such as text generation, language translation, sentiment analysis, and question-answering. You could also highlight the model's accuracy, speed, and efficiency compared to other language models in the market. Additionally, you could demonstrate how the model can be fine-tuned for specific tasks or industries to showcase its versatility and customization options."}

# Memory Types

**ConversationMemory**
- This memory allows for storing of messages and then extracts the messages in a variable

**ConversationBufferWindowMemory**
- This memory keeps a list of the interactions of the conversation over time. It only the last K interactions

**ConversationTokenBufferMemory**
- This memory keeps a buffer of recent interactions on memory, and uses token length rather than number of interactions to determine when to flush interactions.

**ConversationSummaryMemory**
- This memory creates as summary of the conversation over time.

# Additional Memory Types

**Vector data memory**
- Stores text (from conversation or elsewhere) in a vector database and retrieves the most relevant blocks of text.

**Entity memories**
- Using an LLM, it remembers details about specific entities

You can also use multiple memories at one time.
E.g., Conversation memory + Entity memory to recall individuals.

You can also store the conversation in a conventional database (such as key-value store or SQL)